#**Análise Estatística sobre Índice de Desenvolvimento da Educação Básica**
> Andressa Lima Colares - 471151

**Sobre o Ideb**
> O Índice de Desenvolvimento da Educação Básica (Ideb) foi criado em 2007 e reúne, em um só indicador, os resultados de dois conceitos igualmente importantes para a qualidade da educação: o fluxo escolar e as médias de desempenho nas avaliações.

**Cobertura Atemporal do Conjunto**
> 2005 - 2023


**Base de dados**
> A tabela utilizada para essa análisa estatística, é encontrada de forma pública e tratada nesse diretório:


```
 https://basedosdados.org/dataset/96eab476-5d30-459b-82be-f888d4d0d6b9?table=bc84dea9-1126-4423-86d2-8835e6b19a72
```

> A base de dados contém inforamções sobre escolas municipais de educação básica de todo o país.

As colunas são:

1. `ano `- Ano de referência
2. `rede` - Tipo de rede de ensino (total)
3. `ensino` - Nível de ensino (e.g., médio)
4. `anos_escolares`- Intervalo de anos escolares
5. `taxa_aprovacao` - Taxa de aprovação dos estudantes
6. `indicador_rendimento` - Indicador de rendimento
7. `nota_saeb_matematica` - Nota média em matemática no SAEB
8. `nota_saeb_lingua_portuguesa` - Nota média em língua portuguesa no SAEB
9. `nota_saeb_media_padronizada` - Média padronizada das notas do SAEB
10. `ideb` - Índice de Desenvolvimento da Educação Básica
11. `projecao` - Projeção do IDEB para o futuro


#**Passo a passo para a criação de uma rede neural, baseada nesse diretório**

 **Passo 1 - Preparação dos Dados**
* `Normalizar` os valores numéricos para padronizar as escalas.
* `Codificar` as variáveis categóricas (como rede e ensino) em valores numéricos usando One-Hot Encoding.
* `Tratar `valores nulos na coluna projecao.

In [3]:
import pandas as pd

# Carregar o dataset
data = pd.read_csv('/content/br_inep_ideb_brasil.csv.gz')

# Remover linhas onde o IDEB (alvo) está ausente e preencher valores faltantes na projeção com a média
data_cleaned = data.dropna(subset=['ideb'])
data_cleaned['projecao'].fillna(data_cleaned['projecao'].mean(), inplace=True)

# Definir as variáveis de entrada (X) e o alvo (y)
X = data_cleaned.drop(columns=['ideb'])
y = data_cleaned['ideb']


**Passo 2 - Pré-processamento dos Dados**  
  
É necessário `codificar` as colunas categóricas e `padronizar` as colunas numéricas.

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Colunas categóricas e numéricas
categorical_cols = ['rede', 'ensino', 'anos_escolares']
numerical_cols = ['ano', 'taxa_aprovacao', 'indicador_rendimento',
                  'nota_saeb_matematica', 'nota_saeb_lingua_portuguesa',
                  'nota_saeb_media_padronizada', 'projecao']

# Pré-processamento: Padronização para numéricos e OneHot para categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Aplicar pré-processamento
X_transformed = preprocessor.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)


**Passo 3 - Construir a Rede Neural**  
  
Após as outras etapas concluidas, é necessário construir o modelo de `rede neural` usando o `TensorFlow/Keras`.  
  
O modelo terá uma camada de entrada, camadas ocultas com funções de ativação ReLU, e uma camada de saída para prever o Ideb

In [6]:
import tensorflow as tf

# Definir a arquitetura da rede neural
input_shape = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),  # Primeira camada oculta
    tf.keras.layers.Dense(32, activation='relu'),  # Segunda camada oculta
    tf.keras.layers.Dense(1, activation='linear')  # Camada de saída para regressão
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])



**Passo 4 - Treinar o Modelo**

Agora, podemos treinar o modelo com o conjunto de treino.



In [7]:
# Treinamento da rede neural
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)


**Passo 5 - Avaliar o Modelo**  
  
Após o treinamento, é necessário avaliar o desempenho do modelo no conjunto de teste.



In [8]:
# Avaliar o modelo com o conjunto de teste
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

print(f'Mean Absolute Error no conjunto de teste: {test_mae}')


Mean Absolute Error no conjunto de teste: 0.13204656541347504


**Passo 6 - Resumo do modelo**

Por fim, exibir o resumo do modelo para entender a arquitetura final.



In [9]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,797 (38.27 KB)

 Trainable params: 3,265 (12.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,532 (25.52 KB)

# Explicação de cada passo:
* `Carregamento e Limpeza:` Carrega o arquivo CSV, remove valores nulos em ideb e preenche valores ausentes em projecao.
* ` Pré-processamento:` Codifica as variáveis categóricas (rede, ensino, etc.) com One-Hot Encoding e padroniza as numéricas (normalização).
* `Construção da Rede Neural:` Definimos uma rede com 2 camadas ocultas densas com 64 e 32 neurônios, respectivamente. A última camada tem 1 neurônio porque estamos fazendo uma regressão.
* `Treinamento:` O modelo é treinado por 100 épocas, e 20% dos dados de treinamento são usados para validação.
* `Avaliação:` O modelo é avaliado no conjunto de teste para calcular o erro absoluto médio (MAE).
* `Resumo:` O modelo final é apresentado com seu resumo técnico.